In [1]:
import keras
from keras import layers
from keras import models
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from keras.datasets import imdb
from keras.preprocessing import sequence

Using TensorFlow backend.


In [8]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-10-21 00:15:59--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.20.12.96, 52.0.192.127, 52.86.66.223, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.20.12.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  47.3MB/s    in 0.3s    

2019-10-21 00:15:59 (47.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
log_dir = 'my_log_dir'

In [0]:
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(log_dir)
)
# Tunnel port 6006 (TensorBoard assumed running)
get_ipython().system_raw('./ngrok http 6006 &')

In [13]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://f4d92953.ngrok.io


In [0]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

In [0]:
batch_size = 128
num_classes = 10
epochs = 3

In [16]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()


11493376/11490434 [==============================] - 0s 0us/step


In [17]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [0]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
import keras.callbacks
import keras.backend.tensorflow_backend as KTF
import tensorflow as tf

old_session = KTF.get_session()

session = tf.Session('')
KTF.set_session(session)
KTF.set_learning_phase(1)

In [20]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])


In [0]:
tb_cb = keras.callbacks.TensorBoard(log_dir="~/tflog/", histogram_freq=1)
cbks = [tb_cb]

In [23]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks=cbks,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 3s 51us/step - loss: 0.2475 - acc: 0.9234 - val_loss: 0.1175 - val_acc: 0.9646


InvalidArgumentError: ignored

In [0]:
KTF.set_session(old_session)

In [0]:
tb_cb = keras.callbacks.TensorBoard(log_dir="~/tflog/", histogram_freq=1)

In [0]:
max_features = 2000
max_len = 500

In [3]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

17465344/17464789 [==============================] - 0s 0us/step


In [4]:
model = models.Sequential()
model.add(Embedding(max_features, 
                    128, 
                    input_length=max_len,
                    name='embed'))

model.add(Conv1D(32, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(32, 7, activation='relu'))
model.add(Flatten())
model.add(Dense(1))

model.summary()





Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
flatten_1 (Flatten)          (None, 2944)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2945      
Total params: 294,849
Trainable params: 294,849
Non-trainable params: 0
____________________________________________

In [5]:
model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy', 
              metrics=['acc'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
callbacks = [
             keras.callbacks.TensorBoard(
                 log_dir = 'my_log_dir',
                 histogram_freq = 1,
                 embeddings_freq = 1
             )
]

In [7]:
history = model.fit(x_train, 
                    y_train, 
                    epochs=20, 
                    batch_size=128, 
                    validation_split=0.2, 
                    callbacks=callbacks)




Train on 20000 samples, validate on 5000 samples








Epoch 1/20
20000/20000 [==============================] - 6s 325us/step - loss: 0.6354 - acc: 0.6519 - val_loss: 0.4387 - val_acc: 0.8274


ValueError: ignored

In [0]:
!tensorboard --logdir=my_log_dir